<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Notebooks/RoleMappingAspire/Rollen_pro_Abteilung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beschreibung dieses Notebooks

Dieses Notebook wurde entwickelt, um Daten aus einem Google Sheet zu lesen, zu verarbeiten und eine zusammenfassende Pivot-Tabelle zu erstellen. Das Hauptziel ist es, einen Überblick über die "Job Roles" und zugehörigen "People Manager" pro Abteilung zu geben, basierend auf den ersten Zeichen der Abteilungskürzel. Zusätzlich können die angezeigten "Job Roles" über eine separate Filtertabelle im Google Sheet gesteuert werden.

### Funktionsweise Schritt für Schritt:

1.  **Bibliotheken installieren & importieren:** Notwendige Python-Bibliotheken (z.B. `pandas` für Datenanalyse, `gspread` für Google Sheets) werden geladen.
2.  **Authentifizierung bei Google:** Das Notebook authentifiziert sich bei Google Drive und Google Sheets, um auf die benötigten Dokumente zugreifen zu können.
3.  **Google Sheet öffnen:** Das spezifische Google Sheet wird über seine URL geöffnet.
4.  **Daten einlesen:**
    *   Der Reiter "GroupAssignments" wird in einen Pandas DataFrame geladen. Dieser enthält die Kerninformationen zu den "SUPERVISOR ORGANISATION SHORT CODE", "Job Role" und "PEOPLE MANAGER".
    *   Der Reiter "FilterRoles" wird ebenfalls eingelesen. Diese Tabelle dient dazu, festzulegen, welche "Job Roles" in der finalen Ausgabe berücksichtigt werden sollen. Nur "Job Roles", bei denen die Spalte "Role Relevant" auf `TRUE` gesetzt ist, werden verwendet.
5.  **Daten aufbereiten:**
    *   Die relevanten Spalten (`SUPERVISOR ORGANISATION SHORT CODE`, `Job Role`, `PEOPLE MANAGER`) werden ausgewählt und umbenannt.
    *   Die "Job Role"-Spalte, die kommaseparierte Werte enthalten kann, wird in einzelne Zeilen aufgeteilt, sodass jede Zeile eine einzelne "Job Role" repräsentiert.
    *   Ein "SupervisorGroup"-Kürzel wird erstellt, indem die ersten Zeichen des "SUPERVISOR ORGANISATION SHORT CODE" verwendet werden (derzeit auf 1 Zeichen gekürzt, kann aber angepasst werden, um z.B. die ersten 3 oder 5 Zeichen zu nutzen).
    *   Die Daten werden basierend auf den als `TRUE` markierten Rollen aus der "FilterRoles"-Tabelle gefiltert.
6.  **Pivot-Tabelle erstellen:**
    *   Die gefilterten Daten werden nach "SupervisorGroup" und "Job Role" gruppiert, um die Anzahl der Vorkommen jeder "Job Role" pro Gruppe zu zählen.
    *   Diese Gruppierung wird in eine Pivot-Tabelle umgewandelt, bei der die "SupervisorGroup" die Indexzeilen, die "Job Roles" die Spalten und die Anzahlen die Werte bilden.
7.  **People Manager zusammenfassen:**
    *   Für jede "SupervisorGroup" werden die zugehörigen "People Manager" gesammelt und als kommaseparierte Zeichenkette zusammengeführt (Duplikate werden entfernt).
8.  **Finale Tabelle kombinieren & anzeigen:**
    *   Die Pivot-Tabelle wird mit den zusammengefassten "People Manager"-Informationen zusammengeführt.
    *   Die resultierende Tabelle wird angezeigt und enthält die "SupervisorGroup", die "People Manager" und die Zählung der relevanten "Job Roles".
9.  **Ergebnis exportieren:** Die finale Tabelle wird als CSV-Datei (`job_role_summary_filtered.csv`) gespeichert.

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren für Google Drive & Sheets
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen über URL
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8/edit?gid=647371453#gid=647371453"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Reiter "GroupAssignments" einlesen als DataFrame
worksheet = spreadsheet.worksheet("GroupAssignments")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Optional: Erste Zeilen anschauen
print("Datenvorschau:")
display(df.head())

# Schritt 5: Relevante Spalten auswählen
df_relevant = df[['SUPERVISOR ORGANISATION SHORT CODE', 'Job Role']].copy()
df_relevant.columns = ['SupervisorShortCode', 'JobRole']

# Schritt 6: JobRoles splitten und Zeilen expandieren
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# NEU: Kürzen der Abteilungskürzel auf die ersten 5 Zeichen zur Gruppierung
df_exploded['SupervisorGroup'] = df_exploded['SupervisorShortCode'].astype(str).str[:5]

# Schritt 7 (neu): Gruppieren und Zählen nach gekürztem Gruppenkürzel und JobRole
df_grouped = df_exploded.groupby(['SupervisorGroup', 'JobRole']).size().reset_index(name='Count')

# Schritt 8 (neu): Pivot-Tabelle erstellen mit zusammengefassten Gruppen
df_pivot = df_grouped.pivot(index='SupervisorGroup', columns='JobRole', values='Count').fillna(0).astype(int)

# Optional: Sortieren für Übersichtlichkeit
df_pivot = df_pivot.sort_index()
df_pivot = df_pivot.reindex(sorted(df_pivot.columns), axis=1)

# Schritt 9 bleibt gleich
print("Pivot-Tabelle mit gruppierten Abteilungskürzeln:")
display(df_pivot)

# Optional: Exportieren
df_pivot.to_csv("job_role_summary.csv")
print("CSV gespeichert als 'job_role_summary.csv'")

in spalte B der ausgabetabelle sollen noch die "People Manager" aus SPalte "E" der source Tabelle aufgelistet werden. Kommagetrennt falls mehrfachnennung. ergänze oder ändere den python code für colab

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren für Google Drive & Sheets
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen über URL
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8/edit?gid=647371453#gid=647371453"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Reiter "GroupAssignments" einlesen als DataFrame
worksheet = spreadsheet.worksheet("GroupAssignments")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Optional: Erste Zeilen anschauen
print("Datenvorschau:")
display(df.head())

# Schritt 5: Relevante Spalten auswählen
df_relevant = df[['SUPERVISOR ORGANISATION SHORT CODE', 'Job Role']].copy()
df_relevant.columns = ['SupervisorShortCode', 'JobRole']

# Schritt 6: JobRoles splitten und Zeilen expandieren
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# NEU: Kürzen der Abteilungskürzel auf die ersten 5 Zeichen zur Gruppierung
df_exploded['SupervisorGroup'] = df_exploded['SupervisorShortCode'].astype(str).str[:5]

# Schritt 6: JobRoles splitten und Zeilen expandieren
df_relevant = df[['SUPERVISOR ORGANISATION SHORT CODE', 'Job Role', 'PEOPLE MANAGER']].copy()
df_relevant.columns = ['SupervisorShortCode', 'JobRole', 'PeopleManager']

df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# Schritt 7: Gruppierungskürzel aus SupervisorCode ableiten (ersten 5 Zeichen)
df_exploded['SupervisorGroup'] = df_exploded['SupervisorShortCode'].astype(str).str[:5]

# Schritt 8: Pivot-Tabelle mit aggregierten JobRole-Zählungen
df_grouped = df_exploded.groupby(['SupervisorGroup', 'JobRole']).size().reset_index(name='Count')
df_pivot = df_grouped.pivot(index='SupervisorGroup', columns='JobRole', values='Count').fillna(0).astype(int)

# Schritt 9: People Manager pro SupervisorGroup sammeln (einmalig, kommasepariert)
pm_df = (
    df_exploded[['SupervisorGroup', 'PeopleManager']]
    .dropna()
    .drop_duplicates()
    .groupby('SupervisorGroup')['PeopleManager']
    .apply(lambda x: ', '.join(sorted(set(x))))
    .reset_index()
)

# Schritt 10: Kombinieren der Pivot-Tabelle mit People Managers
df_pivot.reset_index(inplace=True)
df_final = pd.merge(df_pivot, pm_df, on='SupervisorGroup', how='left')

# Optional: Sortieren der Spalten
people_col = df_final.pop('PeopleManager')
df_final.insert(1, 'PeopleManager', people_col)  # Als 2. Spalte einfügen
df_final = df_final.sort_values(by='SupervisorGroup')
df_final = df_final.set_index('SupervisorGroup')

# Ergebnis anzeigen
print("Pivot-Tabelle mit gruppierten Abteilungen & People Managern:")
display(df_final)

# Exportieren als CSV
df_final.to_csv("job_role_summary_with_managers.csv")
print("CSV gespeichert als 'job_role_summary_with_managers.csv'")

ch möchte festlegen können, welche "Job Role" er mir anzeigt. in der gleichen GTabelle hat es einen Reiter "FilterRoles". In Spalte A sind die "Job Role" aufgelistet. Es Sollen nur die Job Roles ausgegeben werden, welche auch in Spalte "B" mit dem Titel "Role Relevant" die Checkbox auf TRUE haben. gebe mit den python code für colab

In [ ]:
# Schritt 1: Bibliotheken installieren und importieren
!pip install --quiet gspread pandas gspread-dataframe google-auth

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth

# Schritt 2: Authentifizieren für Google Drive & Sheets
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

# Schritt 3: Google Sheet öffnen über URL
sheet_url = "https://docs.google.com/spreadsheets/d/1dS9JQGu_w-s7L7Xlu9KmJQpSJDA3IGSXO0DJC_rqyh8/edit?gid=647371453#gid=647371453"
spreadsheet = gc.open_by_url(sheet_url)

# Schritt 4: Reiter "GroupAssignments" einlesen als DataFrame
worksheet = spreadsheet.worksheet("GroupAssignments")
df = get_as_dataframe(worksheet, evaluate_formulas=True)

# Zusätzlicher Schritt: Filtertabelle "FilterRoles" einlesen
worksheet_roles = spreadsheet.worksheet("FilterRoles")
df_roles = get_as_dataframe(worksheet_roles, evaluate_formulas=True)

# Nur relevante Rollen (wo 'Role Relevant' TRUE ist)
role_filter = (
    df_roles[df_roles['Role Relevant'] == True]['Job Role']
    .dropna()
    .astype(str)
    .str.strip()
    .tolist()
)

# Optional: Erste Zeilen anschauen
print("Datenvorschau:")
display(df.head())

# Schritt 5: Relevante Spalten auswählen
df_relevant = df[['SUPERVISOR ORGANISATION SHORT CODE', 'Job Role']].copy()
df_relevant.columns = ['SupervisorShortCode', 'JobRole']

# Schritt 6: JobRoles splitten und Zeilen expandieren
df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# NEU: Kürzen der Abteilungskürzel auf die ersten 5 Zeichen zur Gruppierung
df_exploded['SupervisorGroup'] = df_exploded['SupervisorShortCode'].astype(str).str[:3]

# Schritt 6: JobRoles splitten und Zeilen expandieren
df_relevant = df[['SUPERVISOR ORGANISATION SHORT CODE', 'Job Role', 'PEOPLE MANAGER']].copy()
df_relevant.columns = ['SupervisorShortCode', 'JobRole', 'PeopleManager']

df_relevant['JobRole'] = df_relevant['JobRole'].fillna('').astype(str).str.split(',')
df_exploded = df_relevant.explode('JobRole')
df_exploded['JobRole'] = df_exploded['JobRole'].str.strip()

# Schritt 7: Gruppierungskürzel aus SupervisorCode ableiten (ersten 5 Zeichen)
df_exploded['SupervisorGroup'] = df_exploded['SupervisorShortCode'].astype(str).str[:1] #hinten die Zahl ändern für Kürzel - wie mit Abt verbinden

# Schritt 8: "FilterRoles"-Tabelle einlesen und auf TRUE gesetzte Rollen extrahieren
filter_sheet = spreadsheet.worksheet("FilterRoles")
df_filter = get_as_dataframe(filter_sheet, evaluate_formulas=True)

# Saubere Rollenliste extrahieren
relevant_roles = df_filter[
    df_filter['Role Relevant'].astype(str).str.upper().str.strip() == 'TRUE']['Job Role'].dropna().astype(str).str.strip().tolist()

# Nur relevante Rollen behalten
df_filtered = df_exploded[df_exploded['JobRole'].isin(relevant_roles)].copy()

# Schritt 9: Pivot-Tabelle mit aggregierten JobRole-Zählungen
df_grouped = df_filtered.groupby(['SupervisorGroup', 'JobRole']).size().reset_index(name='Count')
df_pivot = df_grouped.pivot(index='SupervisorGroup', columns='JobRole', values='Count').fillna(0).astype(int)

# Schritt 10: People Manager pro SupervisorGroup sammeln (einmalig, kommasepariert)
pm_df = (
    df_filtered[['SupervisorGroup', 'PeopleManager']]
    .dropna()
    .drop_duplicates()
    .groupby('SupervisorGroup')['PeopleManager']
    .apply(lambda x: ', '.join(sorted(set(x))))
    .reset_index()
)

# Schritt 11: Kombinieren der Pivot-Tabelle mit People Managers
df_pivot.reset_index(inplace=True)
df_final = pd.merge(df_pivot, pm_df, on='SupervisorGroup', how='left')

# Optional: Sortieren der Spalten
people_col = df_final.pop('PeopleManager')
df_final.insert(1, 'PeopleManager', people_col)  # Als 2. Spalte einfügen
df_final = df_final.sort_values(by='SupervisorGroup')
df_final = df_final.set_index('SupervisorGroup')

# Ergebnis anzeigen
print("Pivot-Tabelle mit gefilterten Job Roles & People Managern:")
display(df_final)

# Exportieren als CSV
df_final.to_csv("job_role_summary_filtered.csv")
print("CSV gespeichert als 'job_role_summary_filtered.csv'")